In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
# multivariate linear regression with regularization
from sklearn.linear_model import LinearRegression, Ridge, Lasso
# support vector machine regression
from sklearn.svm import SVR
from sklearn.metrics import r2_score
# neural network
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout, Bidirectional, BatchNormalization
# normalization
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.pipeline import Pipeline
# score
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, GridSearchCV, RandomizedSearchCV
# import keras
import tensorflow.keras
from tensorflow.keras import optimizers, regularizers
from tensorflow.keras.layers import Activation
from tensorflow.keras import backend as K
# import regularizer
from tensorflow.keras.regularizers import l1, l2
# import matplotlib
import matplotlib.pyplot as plt
# os
import os
import pickle
from calendar import monthrange

In [2]:
area_df     = pd.read_excel('area_2003-2019.xls', sheet_name = 'Sheet1')
lychee_yield_df = pd.read_excel('lycheeproduct.xlsx')
humidChiangrai=pd.read_excel('humidchiangrai.xlsx',sheet_name = 'Sheet2')
tempchiangrai=pd.read_excel('temperaturechiangrai.xlsx',sheet_name = 'Sheet2')
rainchiangrai=pd.read_excel('rainchiangrai.xlsx')
tempchiangrai=tempchiangrai.iloc[:,0:12]
humidChiangrai=humidChiangrai.iloc[:,0:12]
print(tempchiangrai)

JAN   FEB   MAR   APR   MAY   JUN   JUL   AUG   SEP   OCT   NOV   DEC
0   19.9  21.0  24.2  27.0  26.6  27.4  26.8  27.0  26.4  24.9  21.4  20.5
1   20.2  21.7  24.8  27.2  26.6  27.2  26.3  26.3  26.8  25.9  22.1  19.1
2   19.5  22.8  24.7  27.1  27.5  27.0  27.2  27.0  27.1  26.4  23.0  20.5
3   21.9  22.3  24.5  28.5  29.0  28.6  27.4  26.7  26.7  25.9  22.6  21.8
4   20.8  22.0  23.6  25.7  26.7  27.4  27.2  26.3  26.7  25.6  22.4  20.3
5   20.6  22.8  24.6  27.5  27.4  27.6  26.5  26.9  26.5  26.1  25.2  22.1
6   21.1  24.1  24.2  28.5  28.2  27.6  26.8  26.7  26.3  24.5  24.0  18.5
7   19.3  22.0  24.6  27.3  27.9  28.0  26.9  26.5  26.8  25.8  24.1  20.9
8   19.6  21.7  25.1  26.4  28.5  28.7  27.7  27.2  27.2  25.6  24.6  21.7
9   19.0  21.6  26.1  29.5  28.9  27.5  26.8  27.1  27.1  26.5  24.6  21.7
10  21.5  22.8  25.9  26.7  27.6  27.8  26.8  27.1  27.0  26.1  24.2  20.7
11  21.0  22.6  24.7  25.9  27.0  27.1  27.3  26.8  27.3  26.1  23.1  22.5


In [3]:
rainchiangrai=pd.read_excel('cm_met_ChiangRai.xlsx',sheet_name='Rainfall')
pressurechiangrai=pd.read_excel('cm_met_ChiangRai.xlsx',sheet_name='Pressure')
pheno=pd.read_excel('cm_met_Chiangrai.xlsx',sheet_name='pheno')

rainchiangrai=rainchiangrai.iloc[0,1:]
rainchiangrai=rainchiangrai.to_numpy()
rainchiangrai=rainchiangrai.reshape(12,13)
rainchiangrai=np.delete(rainchiangrai,12,1)
###############################################3
pressurechiangrai=pressurechiangrai.iloc[0,1:]
pressurechiangrai=pressurechiangrai.to_numpy()
pressurechiangrai=pressurechiangrai.reshape(12,13)
pressurechiangrai=np.delete(pressurechiangrai,12,1)
#########################################################
pheno=pheno.iloc[3,2:]
pheno=pheno.to_numpy()
pheno=pheno.reshape(12,12)
########################################################

inputchiangrai=pd.read_excel('input.xlsx')
Y_chiangrai=inputchiangrai.iloc[3:,3]
inputchiangrai=inputchiangrai.iloc[3:,:4]
inputchiangrai=inputchiangrai.to_numpy()
#######################################################################
Y_chiangrai=Y_chiangrai.to_numpy()
tempchiangrai=tempchiangrai.to_numpy()

humidChiangrai=humidChiangrai.to_numpy()

print(pressurechiangrai.shape)

(12, 12)


In [4]:
####################DATAINPUT##########################################33
X_chiangrai=np.concatenate((tempchiangrai[:,0:2],inputchiangrai[:,0:3]),axis=1)
X_chiangrai=np.concatenate((X_chiangrai,pheno[:,0:3]),axis=1)
#X_chiangrai=np.concatenate((X_chiangrai,rainchiangrai[:,0:2]),axis=1)

In [5]:
def rmse(y_true, y_pred):

    ind_ignorezero = (y_true != 0).reshape(-1,)
    error = (y_true - y_pred)
    se = error**2
    mse = np.mean(se)
    rmse = mse**0.5
    return rmse
def mape(y_true, y_pred):
    error = y_true - y_pred
    pe     = (y_true - y_pred)/y_true*100
    ape = np.abs(pe)
    mape = np.mean(ape)

    return mape  
def mae(y_true, y_pred):

    error = y_true - y_pred
    ae = np.abs(error)
    mae = np.mean(ae)

    return mae

In [6]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = check_arrays(y_true, y_pred)

    ## Note: does not handle mix 1d representation
    #if _is_1d(y_true): 
    #    y_true, y_pred = _check_1d_array(y_true, y_pred)

    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [7]:
n_size=X_chiangrai.shape[0]

X_train, X_test, Y_train, Y_test =  train_test_split(X_chiangrai, Y_chiangrai, test_size=0.3)
X_train_ord = X_chiangrai[:int(n_size*(1-0.3)), :]
X_test_ord  = X_chiangrai[int(n_size*(1-0.3)):, :]
Y_train_ord = Y_chiangrai[:int(n_size*(1-0.3))]
Y_test_ord  = Y_chiangrai[int(n_size*(1-0.3)):]
#####################################################3



In [8]:
train_mae = np.array([])
test_mae = np.array([])

train_rmse = np.array([])
test_rmse = np.array([])
train_mape = np.array([])
test_mape = np.array([])

lin_reg = LinearRegression()
lin_reg.fit(X_train, np.log1p(Y_train))
Y_all_test = lin_reg.predict(X_chiangrai)
Y_all_test[Y_all_test<0] = 0

Y_all_test = np.expm1(Y_all_test)

train_mae = mae(Y_chiangrai[:int(n_size*(1-0.3))], Y_all_test[:int(n_size*(1-0.3))])
test_mae = mae(Y_chiangrai[int(n_size*(1-0.3)):], Y_all_test[int(n_size*(1-0.3)):])

train_rmse = rmse(Y_chiangrai[:int(n_size*(1-0.3))], Y_all_test[:int(n_size*(1-0.3))])
test_rmse = rmse(Y_chiangrai[int(n_size*(1-0.3)):], Y_all_test[int(n_size*(1-0.3)):])

train_mape = mape(Y_chiangrai[:int(n_size*(1-0.3))], Y_all_test[:int(n_size*(1-0.3))])
test_mape = mape(Y_chiangrai[int(n_size*(1-0.3)):], Y_all_test[int(n_size*(1-0.3)):])

print('train MAE {:.2f}\ttest MAE {:.2f}'.format(train_mae, test_mae))
print('train RMSE {:.2f}\ttest RMSE {:.2f}'.format(train_rmse, test_rmse))
print('train MAPE {:.2f}\ttest MAPE {:.2f}'.format(train_mape, test_mape))
print('=========================')





train MAE 948.15	test MAE 263.07
train RMSE 2088.35	test RMSE 419.75
train MAPE 12.07	test MAPE 4.69


In [9]:
#################################Chiangmai###########################
inputchiagmai=pd.read_excel('inputchiangmai.xlsx')
tempchiangmai=pd.read_excel('cm_met_Chiangmai.xlsx',sheet_name='Temperature')
phenochiangmai=pd.read_excel('cm_met_Chiangmai.xlsx',sheet_name='Phenomena')
humidchiangmai=pd.read_excel('cm_met_Chiangmai.xlsx',sheet_name='Humidity')
Y_chiangmai=inputchiagmai.iloc[1:,3]

In [10]:
tempchiangmai=tempchiangmai.iloc[4,1:]
tempchiangmai=tempchiangmai.to_numpy()
tempchiangmai=tempchiangmai.reshape(13,13)
tempchiangmai=np.delete(tempchiangmai,9,0)
inputchiagmai=inputchiagmai.iloc[1:,:3]
#######################################################
phenochiangmai=phenochiangmai.iloc[3,1:]
phenochiangmai=phenochiangmai.to_numpy()
phenochiangmai=phenochiangmai.reshape(13,13)
phenochiangmai=np.delete(phenochiangmai,9,0)

In [12]:
inputchiagmai=inputchiagmai.to_numpy()
print(inputchiagmai)

[[  231    60 57865]
 [  131    59 60213]
 [  162    61 54641]
 [  118    96 54641]
 [  159    16 57652]
 [  135    62 59709]
 [   90    88 57927]
 [  184    70 58294]
 [  146    62 58046]
 [  159   116 53477]
 [   88    70 52982]
 [  109    41 47254]]


In [11]:
Y_chiangmai=Y_chiangmai.to_numpy()


In [13]:
X_chiangmai=np.concatenate((tempchiangmai[:,0:2],inputchiagmai[:,0:3]),axis=1)

X_chiangmai=np.concatenate((X_chiangmai,phenochiangmai[:,0:3]),axis=1)


In [14]:
n_size=X_chiangrai.shape[0]

X_train, X_test, Y_train, Y_test =  train_test_split(X_chiangmai, Y_chiangmai, test_size=0.3)
X_train_ord = X_chiangmai[:int(n_size*(1-0.3)), :]
X_test_ord  = X_chiangmai[int(n_size*(1-0.3)):, :]
Y_train_ord = Y_chiangmai[:int(n_size*(1-0.3))]
Y_test_ord  = Y_chiangmai[int(n_size*(1-0.3)):]

In [15]:
lin_reg = LinearRegression()
lin_reg.fit(X_train, np.log1p(Y_train))
Y_all_test = lin_reg.predict(X_chiangmai)
Y_all_test[Y_all_test<0] = 0

Y_all_test = np.expm1(Y_all_test)

train_mae = mae(Y_chiangmai[:int(n_size*(1-0.3))], Y_all_test[:int(n_size*(1-0.3))])
test_mae = mae(Y_chiangmai[int(n_size*(1-0.3)):], Y_all_test[int(n_size*(1-0.3)):])

train_rmse = rmse(Y_chiangmai[:int(n_size*(1-0.3))], Y_all_test[:int(n_size*(1-0.3))])
test_rmse = rmse(Y_chiangmai[int(n_size*(1-0.3)):], Y_all_test[int(n_size*(1-0.3)):])

train_mape = mape(Y_chiangmai[:int(n_size*(1-0.3))], Y_all_test[:int(n_size*(1-0.3))])
test_mape = mape(Y_chiangmai[int(n_size*(1-0.3)):], Y_all_test[int(n_size*(1-0.3)):])

print('train MAE {:.2f}\ttest MAE {:.2f}'.format(train_mae, test_mae))
print('train RMSE {:.2f}\ttest RMSE {:.2f}'.format(train_rmse, test_rmse))
print('train MAPE {:.2f}\ttest MAPE {:.2f}'.format(train_mape, test_mape))
print('=========================')

train MAE 9489.01	test MAE 5069.93
train RMSE 17378.91	test RMSE 10139.86
train MAPE 56.68	test MAPE 20.00


In [16]:
#########################################Neaural Network#########################################

In [17]:
print(X_chiangmai.shape[0])

12


In [18]:
scaler=MinMaxScaler()
reg = l1(0.01)
neural = Sequential()
scaler=MinMaxScaler()
    # input layer
neural.add(Dense(256, activation='relu', input_dim=X_chiangmai.shape[1]))
    # hidden layer
neural.add(Dense(512, activation='relu',kernel_regularizer=reg)) 
neural.add(Dropout(0.4))
neural.add(Dense(512, activation='relu',kernel_regularizer=reg))   
neural.add(Dropout(0.4)) 
neural.add(Dense(512, activation='relu',kernel_regularizer=reg))   
neural.add(Dropout(0.4))     
# output layer
neural.add(Dense(1, activation='relu'))
neural.compile(optimizer='adam', loss='mse',metrics=['mse', 'mae', 'mape'])

In [19]:
n_size=X_chiangrai.shape[0]

X_train, X_test, Y_train, Y_test =  train_test_split(X_chiangrai, Y_chiangrai, test_size=0.3)
X_train_ord = X_chiangrai[:int(n_size*(1-0.3)), :]
X_test_ord  = X_chiangrai[int(n_size*(1-0.3)):, :]
Y_train_ord = Y_chiangrai[:int(n_size*(1-0.3))]
Y_test_ord  = Y_chiangrai[int(n_size*(1-0.3)):]

In [20]:
scaler.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [104]:
X_train=K.constant(X_train)
Y_train=K.constant(Y_train)

In [105]:

neural.fit(X_train,Y_train,epochs=100, verbose=0, batch_size=32)

In [106]:
X_chiangrai=K.constant(X_chiangrai)

In [107]:
y_pred=neural.predict(X_chiangrai)


In [36]:
accuracy=neural.evaluate(X_train, Y_train)

8/1 [================================================================================================================================================================================================================================================] - 0s 1000us/sample - loss: 4461783.0000 - mse: 4461623.0000 - mae: 1469.5554 - mape: 13.3708


In [34]:
X_test=K.constant(X_test)
Y_test=K.constant(Y_test)

In [37]:
accuracy=neural.evaluate(X_test, Y_test)

4/1 [========================================================================================================================] - 0s 2ms/sample - loss: 13814280.0000 - mse: 13814120.0000 - mae: 2293.3701 - mape: 15.1822


In [39]:
accuracy=neural.evaluate(X_chiangrai, Y_chiangrai)


12/1 [========================================================================================================================================================================================================================================================================================================================================================================] - 0s 20ms/sample - loss: 7579282.0000 - mse: 7579122.0000 - mae: 1744.1602 - mape: 13.9746


In [41]:
n_size=X_chiangmai.shape[0]

X_train, X_test, Y_train, Y_test =  train_test_split(X_chiangmai, Y_chiangmai, test_size=0.3)
X_train_ord = X_chiangmai[:int(n_size*(1-0.3)), :]
X_test_ord  = X_chiangmai[int(n_size*(1-0.3)):, :]
Y_train_ord = Y_chiangmai[:int(n_size*(1-0.3))]
Y_test_ord  = Y_chiangmai[int(n_size*(1-0.3)):]

In [42]:
X_train=K.constant(X_train)
Y_train=K.constant(Y_train)
X_test=K.constant(X_test)
Y_test=K.constant(Y_test)
X_chiangmai=K.constant(X_chiangmai)




In [98]:
scaler=MinMaxScaler()
reg = l1(0.01)
neuralchiangmai = Sequential()
scaler=MinMaxScaler()
    # input layer
neuralchiangmai.add(Dense(256, activation='relu', input_dim=X_chiangmai.shape[1]))
    # hidden layer
neuralchiangmai.add(Dense(512, activation='relu',kernel_regularizer=reg)) 
neuralchiangmai.add(Dropout(0.4))
neuralchiangmai.add(Dense(512, activation='relu',kernel_regularizer=reg))   
neuralchiangmai.add(Dropout(0.4)) 
#neuralchiangmai.add(Dense(512, activation='relu',kernel_regularizer=reg))   
#neuralchiangmai.add(Dropout(0.4))
     
# output layer
neuralchiangmai.add(Dense(1, activation='relu'))
neuralchiangmai.compile(optimizer='adam', loss='mse',metrics=['mse', 'mae', 'mape'])

In [99]:
scaler.fit(X_train)
neuralchiangmai.fit(X_train,Y_train,epochs=100, verbose=0, batch_size=32)

In [100]:
y_pred=neuralchiangmai.predict(X_chiangmai)
print(y_pred)

[[22817.328]
 [23749.527]
 [21550.156]
 [21552.533]
 [22735.922]
 [23550.762]
 [22850.902]
 [22989.293]
 [22893.594]
 [21091.29 ]
 [20899.02 ]
 [18638.596]]


In [101]:
accuracy=neuralchiangmai.evaluate(X_train, Y_train)

8/1 [================================================================================================================================================================================================================================================] - 0s 22ms/sample - loss: 42600156.0000 - mse: 42600088.0000 - mae: 6016.4736 - mape: 25.4439


In [102]:
accuracy=neuralchiangmai.evaluate(X_test, Y_test)

4/1 [========================================================================================================================] - 0s 2ms/sample - loss: 14187967.0000 - mse: 14187898.0000 - mae: 3179.7075 - mape: 17.2598


In [103]:
accuracy=neuralchiangmai.evaluate(X_chiangmai, Y_chiangmai)

12/1 [========================================================================================================================================================================================================================================================================================================================================================================] - 0s 14ms/sample - loss: 33129424.0000 - mse: 33129354.0000 - mae: 5070.8848 - mape: 22.7159


In [ ]:
#########################################RNN#########################################################33

In [ ]:
rnn = Sequential()
rnn.add(LSTM(10, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
rnn.add(Dropout(0.2))
rnn.add(LSTM(50, activation='relu'))
rnn.add(Dropout(0.2))
rnn.add(Dense(50, activation='relu'))
rnn.add(Dropout(0.2))
rnn.add(Dense(1, activation='relu'))
rnn.compile(optimizer='adam', loss='mse',metrics=['mse', 'mae', 'mape'])
